In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
anime_data = pd.read_csv("data/anime.csv")
anime_user_ratings = pd.read_csv("data/animelist.csv")
synopsis_df = pd.read_csv("data/anime_with_synopsis.csv")
anime_data = anime_data.rename(columns = {"MAL_ID": "anime_id"})
anime_id_to_name = anime_data[["anime_id", "Name"]]

In [3]:
ratings_df = anime_user_ratings.merge(anime_id_to_name, left_on = 'anime_id', right_on = 'anime_id', how = 'left')
ratings_df = ratings_df[["user_id", "Name", "anime_id","rating","watching_status"]]
ratings_df.head()

,user_id,Name,anime_id,rating,watching_status
0,0,Basilisk: Kouga Ninpou Chou,67,9,1
1,0,Fairy Tail,6702,7,1
2,0,Gokusen,242,10,1
3,0,Kuroshitsuji,4898,0,1
4,0,One Piece,21,10,1


In [4]:
n_ratings_by_user = ratings_df['user_id'].value_counts()
n_ratings_per_anime = ratings_df['anime_id'].value_counts()
#limit recommendations to those by users who've recommended 100+ anime and anime with over 20000 ratings, and to those either completed or dropped
ratings_df = ratings_df[ratings_df["watching_status"].isin([1,2])]
ratings_df = ratings_df[ratings_df['user_id'].isin(n_ratings_by_user[n_ratings_by_user >= 100].index)].copy() 
ratings_df = ratings_df[ratings_df['anime_id'].isin(n_ratings_per_anime[n_ratings_per_anime >= 20000].index)].copy() 

ratings_df.head()

,user_id,Name,anime_id,rating,watching_status
75,1,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,9989,0,1
76,1,Ansatsu Kyoushitsu,24833,0,1
77,1,Bakuman.,7674,7,1
78,1,Black Clover,34572,0,1
79,1,Boruto: Naruto Next Generations,34566,7,1


In [5]:
Avg_Rating = np.mean(ratings_df['rating'])
print("Average Rating: {} \nNumber of Ratings: {} \nUnique Anime: {}\nUnique Users: ".format(Avg_Rating, len(ratings_df), ratings_df['anime_id'].nunique()), ratings_df['user_id'].nunique())

Average Rating: 6.3713422487674745 
Number of Ratings: 49095534 
Unique Anime: 1501
Unique Users:  241288


In [6]:
pivottable = ratings_df.pivot_table(index="Name",columns="user_id", values="rating").fillna(0)
pivottable

user_id,1,2,3,4,5,6,7,8,11,12,...,353389,353390,353391,353392,353393,353395,353396,353398,353400,353403
Name,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07-Ghost,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11eyes,0.0,0.0,0.0,0.0,0.0,7.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3-gatsu no Lion,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
ef: A Tale of Melodies.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ef: A Tale of Memories.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [52]:
matrix = csr_matrix(pivottable.values)
model = NearestNeighbors(metric="cosine", algorithm= 'brute')
model.fit(matrix)
pd.set_option('display.max_colwidth', None)

def predict():
    random_anime = np.random.choice(pivottable.shape[0]) # This will choose a random anime for the model to make recommendations on
    query = pivottable.iloc[random_anime, :].values.reshape(1, -1)
    distance, suggestions = model.kneighbors(query, n_neighbors=6)
    
    data = []
    for i in range(0, len(distance.flatten())):
        if i == 0:
            print("Suggestions for {}".format(pivottable.index[random_anime]))
        else:
            details = []
            details.append(pivottable.index[suggestions.flatten()[i]])
            details.append(synopsis_df[synopsis_df.Name == pivottable.index[suggestions.flatten()[i]]].sypnopsis.values[0])
            details.append(round(distance.flatten()[i], 4))
            data.append(details)
    recommendation_df = pd.DataFrame(data, columns = ["Anime", "Synopsis", "Distance"])
    return recommendation_df

Let's look at some sample predictions

In [53]:
predict()

Suggestions for Kimetsu no Yaiba


,Anime,Synopsis,Distance
0,Boku no Hero Academia 3rd Season,"s summer arrives for the students at UA Academy, each of these superheroes-in-training puts in their best efforts to become renowned heroes. They head off to a forest training camp run by UA's pro heroes, where the students face one another in battle and go through dangerous tests, improving their abilities and pushing past their limits. However, their school trip is suddenly turned upside down when the League of Villains arrives, invading the camp with a mission to capture one of the students. Boku no Hero Academia 3rd Season follows Izuku ""Deku"" Midoriya, an ambitious student training to achieve his dream of becoming a hero similar to his role model—All Might. Being one of the students caught up amidst the chaos of the villain attack, Deku must take a stand with his classmates and fight for their survival.",0.3034
1,Yakusoku no Neverland,"Surrounded by a forest and a gated entrance, the Grace Field House is inhabited by orphans happily living together as one big family, looked after by their ""Mama,"" Isabella. Although they are required to take tests daily, the children are free to spend their time as they see fit, usually playing outside, as long as they do not venture too far from the orphanage—a rule they are expected to follow no matter what. However, all good times must come to an end, as every few months, a child is adopted and sent to live with their new family, never to be heard from again. However, the three oldest siblings have their suspicions about what is actually happening at the orphanage, and they are about to discover the cruel fate that awaits the children living at Grace Field, including the twisted nature of their beloved Mama.",0.3067
2,Shingeki no Kyojin Season 3 Part 2,"Seeking to restore humanity's diminishing hope, the Survey Corps embark on a mission to retake Wall Maria, where the battle against the merciless ""Titans"" takes the stage once again. Returning to the tattered Shiganshina District that was once his home, Eren Yeager and the Corps find the town oddly unoccupied by Titans. Even after the outer gate is plugged, they strangely encounter no opposition. The mission progresses smoothly until Armin Arlert, highly suspicious of the enemy's absence, discovers distressing signs of a potential scheme against them. Shingeki no Kyojin Season 3 Part 2 follows Eren as he vows to take back everything that was once his. Alongside him, the Survey Corps strive—through countless sacrifices—to carve a path towards victory and uncover the secrets locked away in the Yeager family's basement.",0.3133
3,Shingeki no Kyojin Season 3,"Still threatened by the ""Titans"" that rob them of their freedom, mankind remains caged inside the two remaining walls. Efforts to eradicate these monsters continue; however, threats arise not only from the Titans beyond the walls, but from the humans within them as well. After being rescued from the Colossal and Armored Titans, Eren Yaeger devotes himself to improving his Titan form. Krista Lenz struggles to accept the loss of her friend, Captain Levi chooses Eren and his friends to form his new personal squad, and Commander Erwin Smith recovers from his injuries. All seems well for the soldiers, until the government suddenly demands custody of Eren and Krista. The Survey Corps' recent successes have drawn attention, and a familiar face from Levi's past is sent to collect the wanted soldiers. Sought after by the government, Levi and his new squad must evade their adversaries in hopes of keeping Eren and Krista safe. In Shingeki no Kyojin Season 3 , Eren and his fellow soldiers are not only fighting for their survival against the terrifying Titans, but also against the terror of a far more conniving foe: humans.",0.3147
4,Boku no Hero Academia 2nd Season,"UA Academy, not even a violent attack can disrupt their most prestigious event: the school sports festival. Renowned across Japan, this festival is an opportunity for 

In [65]:
predict()

Suggestions for Code Geass: Hangyaku no Lelouch


,Anime,Synopsis,Distance
0,Code Geass: Hangyaku no Lelouch R2,"One year has passed since the Black Rebellion, a failed uprising against the Holy Britannian Empire led by the masked vigilante Zero, who is now missing. At a loss without their revolutionary leader, Area 11's resistance group—the Black Knights—find themselves too powerless to combat the brutality inflicted upon the Elevens by Britannia, which has increased significantly in order to crush any hope of a future revolt. Lelouch Lamperouge, having lost all memory of his double life, is living peacefully alongside his friends as a high school student at Ashford Academy. His former partner C.C., unable to accept this turn of events, takes it upon herself to remind him of his past purpose, hoping that the mastermind Zero will rise once again to finish what he started, in this thrilling conclusion to the series.",0.0638
1,Death Note,"shinigami, as a god of death, can kill any person—provided they see their victim's face and write their victim's name in a notebook called a Death Note. One day, Ryuk, bored by the shinigami lifestyle and interested in seeing how a human would use a Death Note, drops one into the human realm. High school student and prodigy Light Yagami stumbles upon the Death Note and—since he deplores the state of the world—tests the deadly notebook by writing a criminal's name in it. When the criminal dies immediately following his experiment with the Death Note, Light is greatly surprised and quickly recognizes how devastating the power that has fallen into his hands could be. With this divine capability, Light decides to extinguish all criminals in order to build a new world where crime does not exist and people worship him as a god. Police, however, quickly discover that a serial killer is targeting criminals and, consequently, try to apprehend the culprit. To do this, the Japanese investigators count on the assistance of the best detective in the world: a young and eccentric man known only by the name of L.",0.2833
2,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, something of equal value must be lost."" Alchemy is bound by this Law of Equivalent Exchange—something the young brothers Edward and Alphonse Elric only realize after attempting human transmutation: the one forbidden act of alchemy. They pay a terrible price for their transgression—Edward loses his left leg, Alphonse his physical body. It is only by the desperate sacrifice of Edward's right arm that he is able to affix Alphonse's soul to a suit of armor. Devastated and alone, it is the hope that they would both eventually return to their original bodies that gives Edward the inspiration to obtain metal limbs called ""automail"" and become a state alchemist, the Fullmetal Alchemist. Three years of searching later, the brothers seek the Philosopher's Stone, a mythical relic that allows an alchemist to overcome the Law of Equivalent Exchange. Even with military allies Colonel Roy Mustang, Lieutenant Riza Hawkeye, and Lieutenant Colonel Maes Hughes on their side, the brothers find themselves caught up in a nationwide conspiracy that leads them not only to the true nature of the elusive Philosopher's Stone, but their country's murky history as well. In between finding a serial killer and racing against time, Edward and Alphonse must ask themselves if what they are doing will make them human again... or take away their humanity.",0.3262
3,Shingeki no Kyojin,"Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out of pleasure. To ensure their survival, the remnants of humanity began living within defensive barriers, resulting in one hundred years without a single titan encounter. However, that fragile calm is soon shattered when a col

In [80]:
predict()

Suggestions for Nisemonogatari


,Anime,Synopsis,Distance
0,Nekomonogatari: Kuro,"fter surviving a vampire attack, Koyomi Araragi notices that his friend and savior, Tsubasa Hanekawa, has been acting strange. When he happens to cross paths with her on his way to a bookstore and sees she has a bandage on her face, he knows something must definitely be wrong. Araragi wants to help her, but Hanekawa assures him that her wound is just something she received at home and that he should not concern himself with it. But when a white cat with no tail is hit and killed by a car, the pair bury the creature and the real trouble begins. When Araragi later pays a visit to his friend Meme Oshino and recounts the day's events, he is informed what they have buried is actually an apparition, one perfect for Hanekawa in her current state. Tasked with finding his friend to confirm her safety, he discovers that she has attacked her parents, possessed by the ""Sawari Neko."" Now, it is up to Araragi to help Hanekawa as she once helped him.",0.1641
1,Bakemonogatari,"Koyomi Araragi, a third-year high school student, manages to survive a vampire attack with the help of Meme Oshino, a strange man residing in an abandoned building. Though being saved from vampirism and now a human again, several side effects such as superhuman healing abilities and enhanced vision still remain. Regardless, Araragi tries to live the life of a normal student, with the help of his friend and the class president, Tsubasa Hanekawa. When fellow classmate Hitagi Senjougahara falls down the stairs and is caught by Araragi, the boy realizes that the girl is unnaturally weightless. Despite Senjougahara's protests, Araragi insists he help her, deciding to enlist the aid of Oshino, the very man who had once helped him with his own predicament. Through several tales involving demons and gods, Bakemonogatari follows Araragi as he attempts to help those who suffer from supernatural maladies.",0.1777
2,Monogatari Series: Second Season,"pparitions, oddities, and gods continue to manifest around Koyomi Araragi and his close-knit group of friends: Tsubasa Hanekawa, the group's modest genius; Shinobu Oshino, the resident doughnut-loving vampire; athletic deviant Suruga Kanbaru; bite-happy spirit Mayoi Hachikuji; Koyomi's cutesy stalker Nadeko Sengoku; and Hitagi Senjogahara, the poignant tsundere. Monogatari Series: Second Season revolves around these individuals and their struggle to overcome the darkness that is rapidly approaching. A new semester has begun and with graduation looming over Araragi, he must quickly decide the paths he will walk, as well as the relationships and friends that he'll save. But as strange events begin to unfold, Araragi is nowhere to be found, and a vicious tiger apparition has appeared in his absence. Hanekawa has become its target, and she must fend for herself—or bow to the creature's perspective on the feebleness of humanity.",0.1859
3,Tsukimonogatari,"Koyomi Araragi is studying hard in preparation for his college entrance exams when he begins to notice something very strange: his reflection no longer appears in a mirror, a characteristic of a true vampire. Worried about the state of his body, he enlists the help of the human-like doll Yotsugi Ononoki and her master Yozuru Kagenui, an immortal oddity specialist. Quickly realizing what is wrong with him, Yozuru gives him two choices: either abstain from using the vampiric abilities he received from Shinobu Oshino, or lose his humanity forever.",0.2936
4,Hanamonogatari,"Now that Koyomi Araragi and Hitagi Senjougahara have graduated, very few familiar faces remain at Naoetsu Private High School, one of them being Kanbaru Suruga, holder of the Monkey's Paw. When she begins to hear talk of a mysterious being known as the ""Devil,"" who will magically solve any problem, she immediately thinks these rumors are about her and decides to investigate. She discovers the Devil is actually Rouka Numachi, a former rival from junior high who is providing